In [1]:
%load_ext autoreload
%autoreload 2
from b_get_data import *
from c_model_related import CustomStandardScaler as Cscl
from c_model_related import Trainer as tr
from c_model_related import metrics_create_df
from c_model_related import scaler_antigo
import joblib
from sklearn.preprocessing import StandardScaler

In [2]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]


feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

# Obter os dataframes dos ficheiros

In [3]:
# Obter o caminho dos ficheiros.
root_dir = os.path.abspath('..')
csv_path = os.path.join(root_dir, 'rawdata')
# Importar o dataset de failures
failures_path = os.path.join(csv_path, 'wind-farm-1-failures-training.csv')
failures_df = get_data(failures_path)
# Importar o dataset de signals
signals_path = os.path.join(csv_path, 'wind-farm-1-signals-training.csv')
signals_df = get_data(signals_path)
# Cortar colunas que não têm valores
cols_to_drop = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']
signals_df = signals_df.drop(columns=cols_to_drop)

In [4]:
# print('002 - Criar o dicionário com os Dataframes originais')
df_dict = {'failures_df':failures_df, 'signals_df':signals_df}

In [5]:
df_dict.keys()

dict_keys(['failures_df', 'signals_df'])

# Criação de datasets de components

In [6]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = component_df_creation(
    signals_df)

In [7]:
comp_df_dict = {
    'df_generator': df_generator,
    'df_hydraulic': df_hydraulic,
    'df_gen_bear': df_gen_bear,
    'df_transformer': df_transformer,
    'df_gearbox': df_gearbox
}

In [8]:
for key in comp_df_dict:
    print(comp_df_dict[key].shape)

(434145, 45)
(434145, 34)
(434145, 45)
(434145, 36)
(434145, 33)


# Cópia dos df para só carregar uma vez no ficheiro

In [9]:
comp_prep_df_dict = comp_df_dict.copy()

# Merge dos DF com o DF de failures

In [10]:
component_list = [
    'GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
    'GEARBOX'
]
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = sig_fail_merge_dfs(sig_df=comp_prep_df_dict[key],
                                                fail_df=failures_df,
                                                component=component_list[i])

In [11]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 46)
(434145, 35)
(434145, 46)
(434145, 37)
(434145, 34)


# Fillna by turbine

In [12]:
turbine_list = ['T11', 'T06', 'T01', 'T09', 'T07']
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = fill_na_by_turbine(comp_prep_df_dict[key],
                                                turbine_list)

C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  index['date'] = index['Timestamp']
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:128: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_ = pd.concat([df_, df_final])
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [13]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 47)
(434145, 36)
(434145, 47)
(434145, 38)
(434145, 35)


In [14]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].isna().sum().value_counts(), key)

0    47
dtype: int64 df_generator
0    36
dtype: int64 df_hydraulic
0    47
dtype: int64 df_gen_bear
0    38
dtype: int64 df_transformer
0    35
dtype: int64 df_gearbox


# Criação de Variável alvo

In [15]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = aplic_var_target(comp_prep_df_dict[key], 60)

In [16]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 48) df_generator
(434145, 37) df_hydraulic
(434145, 48) df_gen_bear
(434145, 39) df_transformer
(434145, 36) df_gearbox


# Feature selection

In [17]:
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = comp_prep_df_dict[key].drop(
        columns=feat_drop_list[i])

In [18]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 31) df_generator
(434145, 26) df_hydraulic
(434145, 31) df_gen_bear
(434145, 28) df_transformer
(434145, 26) df_gearbox


In [19]:
comp_prep_df_dict['df_hydraulic'].columns

Index(['Amb_Temp_Avg', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg', 'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min', 'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg',
       'Blds_PitchAngle_Std', 'Component', 'Cont_Top_Temp_Avg', 'Cont_VCP_ChokcoilTemp_Avg', 'Cont_VCP_WtrTemp_Avg', 'Grd_Prod_VoltPhse1_Avg', 'Grd_Prod_VoltPhse2_Avg', 'Grd_Prod_VoltPhse3_Avg',
       'Grd_RtrInvPhase2_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Nac_Direction_Avg', 'Nac_Temp_Avg', 'Rtr_RPM_Avg', 'Rtr_RPM_Std', 'TTF', 'Timestamp', 'Turbine_ID', '60_days'],
      dtype='object')

# Agrupar pela medida de tempo seleccionada.

In [21]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = group_por_frequency(comp_prep_df_dict[key],
                                                 period='Dia')

In [22]:
comp_prep_df_dict['df_hydraulic'].columns

Index(['Turbine_ID', 'Date', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg', 'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min', 'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg',
       'Blds_PitchAngle_Std', 'Component', 'Grd_Prod_VoltPhse1_Avg', 'Grd_Prod_VoltPhse2_Avg', 'Grd_Prod_VoltPhse3_Avg', 'Nac_Direction_Avg', 'Rtr_RPM_Avg', 'Rtr_RPM_Std', 'TTF', '60_days'],
      dtype='object')

In [20]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 31) df_generator
(3041, 19) df_hydraulic
(3041, 18) df_gen_bear
(3041, 19) df_transformer
(3041, 16) df_gearbox


# Adicionar as medidas de alisamento

In [21]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = add_features(comp_prep_df_dict[key], rolling_win_size=10)

In [23]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 83) df_generator
(3041, 47) df_hydraulic
(3041, 44) df_gen_bear
(3041, 47) df_transformer
(3041, 38) df_gearbox


# Train and test split

In [23]:
df_train_comp_dict = {}
df_test_comp_dict = {}
for key in comp_prep_df_dict:
    df_train_comp_dict[key] = prepare_train_df(comp_prep_df_dict[key], meses = 3)
    df_test_comp_dict[key] = prepare_test_df(comp_prep_df_dict[key], meses = 3)

In [24]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


In [25]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


# Criar o x_train, x_test, y_train e o y_test 

In [26]:
print('012 - separar entre x_train, x_test')
x_train = df_train_comp_dict.copy()
y_train = df_train_comp_dict.copy()
x_test = df_test_comp_dict.copy()
y_test = df_test_comp_dict.copy()

012 - separar entre x_train, x_test


In [27]:
for key in x_train:
    print(x_train[key].shape, key, x_train[key].isna().sum().value_counts())
for key in x_test:
    print(x_test[key].shape, key, x_test[key].isna().sum().value_counts())
for key in y_train:
    print(y_train[key].shape, key, y_train[key].isna().sum().value_counts())
for key in y_test:
    print(y_test[key].shape, key, y_test[key].isna().sum().value_counts())

(2577, 85) df_generator 0    85
dtype: int64
(2576, 49) df_hydraulic 0    49
dtype: int64
(2576, 46) df_gen_bear 0    46
dtype: int64
(2576, 49) df_transformer 0    49
dtype: int64
(2576, 40) df_gearbox 0    40
dtype: int64
(465, 85) df_generator 0    85
dtype: int64
(465, 49) df_hydraulic 0    49
dtype: int64
(465, 46) df_gen_bear 0    46
dtype: int64
(465, 49) df_transformer 0    49
dtype: int64
(465, 40) df_gearbox 0    40
dtype: int64
(2577, 85) df_generator 0    85
dtype: int64
(2576, 49) df_hydraulic 0    49
dtype: int64
(2576, 46) df_gen_bear 0    46
dtype: int64
(2576, 49) df_transformer 0    49
dtype: int64
(2576, 40) df_gearbox 0    40
dtype: int64
(465, 85) df_generator 0    85
dtype: int64
(465, 49) df_hydraulic 0    49
dtype: int64
(465, 46) df_gen_bear 0    46
dtype: int64
(465, 49) df_transformer 0    49
dtype: int64
(465, 40) df_gearbox 0    40
dtype: int64


In [28]:
for key in x_train:
    print(x_train[key].shape, key)
for key in y_train:
    print(y_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)
for key in y_test:
    print(y_test[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


# Model training - variables preparation

In [29]:
print('012 - separar entre x_train, x_test')
# retirar as colunas que não devem entrar no x
cols_to_drop_train = ['Date', 'TTF', '60_days', 'Component']
for key in x_train:
    x_train[key] = x_train[key].drop(columns=cols_to_drop_train)
    x_test[key] = x_test[key].drop(columns=cols_to_drop_train)

012 - separar entre x_train, x_test


In [30]:
print('012.1 - considerar no y set apenas as turbinas que tiveram falhas')
train_turbines = {
    'df_generator': ['T11', 'T06'],
    'df_hydraulic': ['T06', 'T11'],
    'df_gen_bear': ['T07', 'T09'],
    'df_transformer': ['T07'],
    'df_gearbox': ['T09']
}
for key in df_train_comp_dict:
    y_train[key] = y_train[key][y_train[key]['Turbine_ID'].isin(train_turbines[key])]
    x_train[key] = x_train[key][x_train[key]['Turbine_ID'].isin(train_turbines[key])]

012.1 - considerar no y set apenas as turbinas que tiveram falhas


In [31]:
print('xxx - transformar o y de float para int')
col_to_mantain_test = ['60_days']
for key in y_train:
    y_train[key] = y_train[key][col_to_mantain_test].to_numpy().astype(int)[:,0]
    y_test[key] = y_test[key][col_to_mantain_test].to_numpy().astype(int)[:,0]

xxx - transformar o y de float para int


In [32]:
for key in x_train:
    print(x_train[key].shape, key)
for key in y_train:
    print(y_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)
for key in y_test:
    print(y_test[key].shape, key)

(1026, 79) df_generator
(1025, 43) df_hydraulic
(1034, 40) df_gen_bear
(517, 43) df_transformer
(517, 34) df_gearbox
(1026,) df_generator
(1025,) df_hydraulic
(1034,) df_gen_bear
(517,) df_transformer
(517,) df_gearbox
(465, 79) df_generator
(465, 43) df_hydraulic
(465, 40) df_gen_bear
(465, 43) df_transformer
(465, 34) df_gearbox
(465,) df_generator
(465,) df_hydraulic
(465,) df_gen_bear
(465,) df_transformer
(465,) df_gearbox


In [34]:
print('014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro')
print('014.1 - df_generator')
scaler_df_generator = Cscl()
scaler_df_generator.fit(x_train['df_generator'])
# # joblib.dump(scaler_df_generator, 'scaler_df_generator.joblib')
# print('014.2 - df_hydraulic')
# scaler_df_hydraulic = Cscl()
# scaler_df_hydraulic.fit(x_train['df_hydraulic'])
# # joblib.dump(scaler_df_hydraulic, 'scaler_df_hydraulic.joblib')
# print('014.3 - df_gen_bear')
# scaler_df_gen_bear = Cscl()
# scaler_df_gen_bear.fit(x_train['df_gen_bear'])
# # joblib.dump(scaler_df_gen_bear, 'scaler_df_gen_bear.joblib')
# print('014.4 - df_transformer')
# scaler_df_transformer = Cscl()
# scaler_df_transformer.fit(x_train['df_transformer'])
# # joblib.dump(scaler_df_transformer, 'scaler_df_transformer.joblib')
# print('014.5 - df_gearbox')
# scaler_df_gearbox = Cscl()
# scaler_df_gearbox.fit(x_train['df_gearbox'])
# # joblib.dump(scaler_df_gearbox, 'scaler_df_gearbox.joblib')

014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro
014.1 - df_generator


C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\c_model_related.py:23: RuntimeWarning: Mean of empty slice.
  self.mean_[turbine] = X_filter.mean(axis=0)
c:\users\ricar\.venvs\lewagon\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
c:\users\ricar\.venvs\lewagon\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\ricar\.venvs\lewagon\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\ricar\.venvs\lewagon\lib\site-packages\numpy\core\_methods.py:207: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


CustomStandardScaler()

x_train['df_generator'].columns[9]

x_train['df_generator']['Blds_PitchAngle_Std'].isna().sum()

x_train1 = {}
x_test1 = {}

In [ ]:
# def scale(X_train, X_test):
#     X_train1 = X_train.loc[X_train['Turbine_ID']=='T01']
#     X_test1 = X_test.loc[X_test['Turbine_ID']=='T01']
#     X_train1 = X_train1.drop(columns='Turbine_ID')
#     X_test1 = X_test1.drop(columns='Turbine_ID')
    
#     sc = StandardScaler()
#     X_train1 = sc.fit_transform(X_train1)
#     X_test1 = sc.transform(X_test1)
#     turbines = ['T06', 'T07', 'T09', 'T11']
#     for turbine in turbines:
#         X_train_ = X_train.loc[X_train['Turbine_ID']==turbine]
#         X_test_ = X_test.loc[X_test['Turbine_ID']==turbine]
#         X_train_ = X_train_.drop(columns='Turbine_ID')
#         X_test_ = X_test_.drop(columns='Turbine_ID')
     
#         sc = StandardScaler()
#         X_train_ = sc.fit_transform(X_train_)
#         X_test_ = sc.transform(X_test_)
#         X_train1 = np.concatenate((X_train1, X_train_))
#         X_test1 = np.concatenate((X_test1, X_test_))
#     return X_train1, X_test1

In [ ]:
# def scale(X_train, X_test):
#     X_train1 = X_train.loc[X_train['Turbine_ID']=='T01']
#     X_test1 = X_test.loc[X_test['Turbine_ID']=='T01']
#     X_train1 = X_train1.drop(columns='Turbine_ID')
#     X_test1 = X_test1.drop(columns='Turbine_ID')
    
#     sc = StandardScaler()
#     X_train1 = sc.fit_transform(X_train1)
#     X_test1 = sc.transform(X_test1)
#     turbines = ['T06', 'T07', 'T09', 'T11']
#     for turbine in turbines:
#         X_train_ = X_train.loc[X_train['Turbine_ID']==turbine]
#         X_test_ = X_test.loc[X_test['Turbine_ID']==turbine]
#         X_train_ = X_train_.drop(columns='Turbine_ID')
#         X_test_ = X_test_.drop(columns='Turbine_ID')
     
#         sc = StandardScaler()
#         X_train_ = sc.fit_transform(X_train_)
#         X_test_ = sc.transform(X_test_)
#         X_train1 = np.concatenate((X_train1, X_train_))
#         X_test1 = np.concatenate((X_test1, X_test_))
#     return X_train1, X_test1

def fit2(X, y=None):
    mean_ = {}
    std_ = {}

    for turbine in ['T01','T06','T07','T09','T11']:
        X_filter = X[X['Turbine_ID']==turbine]
        X_filter = X_filter.drop(columns='Turbine_ID').to_numpy()
        mean_[turbine] = X_filter.mean(axis=0)
        std_[turbine] = X_filter.std(axis=0)

    return mean_,std_

def transform2(X, y=None):
    X_turbine={}

    for turbine in ['T01','T06','T07','T09','T11']:
        if std_[turbine] == 0:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / 1
        else:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / std_[turbine]

    X_turbine_all=X_turbine['T01']

    for turbine in ['T06','T07','T09','T11']:

        X_turbine_all=np.concatenate([X_turbine_all,X_turbine[turbine]])

    return X_turbine_all

mean_,std_ = fit2(x_train['df_generator'])

std_['T01']

transform(x_train['df_generator'])

In [38]:
print(x_train['df_generator'].shape, x_test['df_generator'].shape,
      y_train['df_generator'].shape, y_test['df_generator'].shape)

(1026, 79) (465, 79) (1026,) (465,)


In [ ]:
print('016 - aplicar o scale com o fit efectuado no teste')
x_train['df_generator'] = scaler_df_generator.transform(x_train['df_generator'])
# x_train['df_hydraulic'] = scaler_df_hydraulic.transform(x_train['df_hydraulic'])
# x_train['df_gen_bear'] = scaler_df_gen_bear.transform(x_train['df_gen_bear'])
# x_train['df_transformer'] = scaler_df_transformer.transform(x_train['df_transformer'])
# x_train['df_gearbox'] = scaler_df_gearbox.transform(x_train['df_gearbox'])

In [33]:
print('016 - aplicar o scale com o fit efectuado no teste')
x_test['df_generator'] = scaler_df_generator.transform(x_test['df_generator'])
# x_test['df_hydraulic'] = scaler_df_hydraulic.transform(x_test['df_hydraulic'])
# x_test['df_gen_bear'] = scaler_df_gen_bear.transform(x_test['df_gen_bear'])
# x_test['df_transformer'] = scaler_df_transformer.transform(x_test['df_transformer'])
# x_test['df_gearbox'] = scaler_df_gearbox.transform(x_test['df_gearbox'])

016 - aplicar o scale com o fit efectuado no teste


NameError: name 'scaler_df_generator' is not defined

In [36]:
gen_scaler = StandardScaler() 

In [39]:
x_train['df_generator'] = x_train['df_generator'].drop(columns=['Turbine_ID'])

In [40]:
gen_scaler.fit(x_train['df_generator'])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [41]:
joblib.dump(gen_scaler, 'gen_scaler.joblib')

['gen_scaler.joblib']

In [42]:
x_train['df_generator'] = gen_scaler.transform(x_train['df_generator'])

In [44]:
x_test['df_generator'] = x_test['df_generator'].drop(columns=['Turbine_ID'])

In [45]:
x_test['df_generator'] = gen_scaler.transform(x_test['df_generator'])

In [40]:
for i in x_train:
    x_train[i], x_test[i] = scaler_antigo(x_train[i], x_test[i])

In [46]:
for key in x_train:
    print(x_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)

(1026, 78) df_generator
(1025, 43) df_hydraulic
(1034, 40) df_gen_bear
(517, 43) df_transformer
(517, 34) df_gearbox
(465, 78) df_generator
(465, 43) df_hydraulic
(465, 40) df_gen_bear
(465, 43) df_transformer
(465, 34) df_gearbox


exp1 = pd.DataFrame(x_test['df_generator'])

x_test['df_generator'][9]

exp1[9]

In [47]:
print('017 - Instanciar e treinar o modelo correspondente')
generator_model = tr(x_train=x_train['df_generator'],
                     y_train=y_train['df_generator'],
                     x_test=x_test['df_generator'],
                     component='df_generator')
generator_model.train()
# joblib.dump(generator_model, 'generator_model.joblib')

017 - Instanciar e treinar o modelo correspondente


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [48]:
print('018 - Obter a previsão para a métrica de poupança')
y_pred_generator = generator_model.predict()

018 - Obter a previsão para a métrica de poupança


In [49]:
print('019 - poupanças')
poupancas_generator, cf_numbers_pred_gen, met_cre_df = metrics_create_df(
    df_test_comp_dict['df_generator'],
    y_test['df_generator'],
    y_pred_generator,
    'df_generator',
    days=15)
print(poupancas_generator, cf_numbers_pred_gen, 'generator')

019 - poupanças
20000.0 {'TP': 1, 'TN': 251, 'FP': 5, 'FN': 0} generator


In [50]:
met_cre_df[met_cre_df['TTF']!=0]

,Date,Turbine_ID,TTF,y_test,y_pred,TP,TN,FP,FN,new_FP
186,2017-06-01,T07,82.0,0,0,0,1,0,0,7.0
187,2017-06-02,T07,81.0,0,0,0,1,0,0,6.0
188,2017-06-03,T07,80.0,0,1,0,0,1,0,7.0
189,2017-06-04,T07,79.0,0,0,0,1,0,0,7.0
190,2017-06-05,T07,78.0,0,0,0,1,0,0,7.0
191,2017-06-06,T07,77.0,0,1,0,0,1,0,7.0
192,2017-06-07,T07,76.0,0,0,0,1,0,0,7.0
193,2017-06-08,T07,75.0,0,0,0,1,0,0,7.0
194,2017-06-09,T07,74.0,0,0,0,1,0,0,7.0
195,2017-06-10,T07,73.0,0,0,0,1,0,0,7.0


In [51]:
y_pred_generator[208:267]

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [52]:
y_pred_generator

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,